In [1]:
# Install the Libraries
!pip install -q langchain torch transformers sentence-transformers datasets faiss-cpu unstructured chromadb gradio langchain-community langchain-huggingface


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
# Import Libraries
from langchain_community.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from transformers import GPT2LMHeadModel, GPT2Tokenizer, pipeline
from langchain import HuggingFacePipeline, LLMChain
from langchain.chains import RetrievalQA, StuffDocumentsChain
from langchain.prompts import PromptTemplate
import gradio as gr
import faiss
import random
import torch
import torch.nn as nn
from transformers import (
    pipeline,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoConfig,
)
from langchain.llms import HuggingFacePipeline

import warnings
from IPython.display import Markdown
import re
import random
import pandas as pd


/usr/local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
E0607 22:39:36.192225254      13 oauth2_credentials.cc:238]            oauth_fetch: UNKNOWN:C-ares status is not ARES_SUCCESS qtype=A name=metadata.google.internal. is_balancer=0: Domain name not found {grpc_status:2, created_time:"2024-06-07T22:39:36.192208099+00:00"}


In [22]:
# # Load Documents from Google Drive
# loader = DirectoryLoader('/kaggle/input/aastu-txt', show_progress = True)
# data = loader.load()



loader = TextLoader("/kaggle/input/agri-txt/LSMS_ESS_16-02-21-1.txt")
data = loader.load()

  0%|          | 0/16 [00:00<?, ?it/s][nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
100%|██████████| 16/16 [00:16<00:00,  1.05s/it]


In [23]:

# Split Documents into Chunks
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    is_separator_regex=True
)
documents = text_splitter.split_documents(data)

print(random.choice(documents).page_content)

Notable contributions include generation of knowledge in the geodynamics of the East African Rift, the petrogenesis of various rocks of Ethiopia, the reconstruction of paleo-environmental setting of Ethiopia, the inventory/baseline survey of earth materials and earth resources of Ethiopia. Researchers in the Departments and graduates from the Departments have been contributing in sectors such as geological surveying, groundwater, construction, energy, mining, and environment. 4.2.2 Program Education Objective (PEO) The program educational objectives describe accomplishments those four years geology program graduates are expected to attain within four years after graduation. Graduates will have applied their expertise to contemporary problem solving, be engaged professionally, and have continued to learn and adapt, and have contributed to their organizations through leadership and teamwork. More specifically, the objectives are expertise, engagement, learning, leadership and teamwork.


In [24]:
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-mpnet-base-v2')

In [25]:
# Create the Vector Store with Chroma
db = FAISS.from_documents(documents, embeddings)

In [38]:
# Test Similarity Search
question = "What is AASTU?"
search_docs = db.similarity_search(question)
print(f"Sample Search Result: {search_docs[0].page_content}")

Sample Search Result: Dear students, welcome to Addis Ababa Science and Technology University (AASTU). For the new students who are joining us this academic year with high performance in national and university entrance exams, congratulations! Established in 2011, AASTU is one of the two science and technology universities in the country. Since its establishment, AASTU has been working to meet the needs of the country in the transformation of its economy focusing in the fields of applied sciences, ICT, and engineering. These disciplines will remain the university’s major strength. AASTU strives to make your campus environment conducive for education, innovation and living. Each one of you will meet students from different parts of the country which will give you an opportunity to learn each other’s languages and cultures. That will be an important experience to work and live in a multicultural society in your future life.


In [40]:
# Query to retrieve similar chunks
query = "The best three departments in AASTU?"

# Retrieve similar chunks based on relevance. We only retrieve 'k' most similar chunks
similar_chunks = db.similarity_search_with_relevance_scores(query, k=3)

# Format document to text format
retrieved_text = [chunk[0].page_content for chunk in similar_chunks]
relevance_score = [chunk[1] for chunk in similar_chunks]

# Store and print as a dataframe
retrieved_chunks = pd.DataFrame(
    list(zip(retrieved_text, relevance_score)),
    columns=["Retrieved Chunks", "Relevance Score"],
)
with pd.option_context("display.max_colwidth", None):
    display(retrieved_chunks)

,Retrieved Chunks,Relevance Score
0,"2.27 Other Services 2.27.1 Social Services The University facilitated for the provision of several social services in the University campus to make your life easy. These include healthcare, restaurants, cafés, shops such as barber, beauty salon, photocopy and stationery and mini supermarkets. In this regard, the university management will continue to exert its effort in extending necessary social and financial services in order to meet your satisfaction. 2.27.2 Counseling Service Gender, Guidance and Counseling, and HIV counseling services are established at office level to provide you support and counseling service in educational, social, psychological and university life issues at large. Please do not hesitate to contact these offices when deemed necessary.\n\n28\n\nAASTU Student Handbook 2023/2024\n\n2.28 Undergraduate Degree Programs\n\nCollege of Engineering\n\nBachelor of science degree in Architecture\n\nBachelor of science degree in Civil Engineering",0.506850
1,"and international learning experiences. Furthermore, a higher\n\nand learning experience, the university has undertaken\n\ndiploma training program equips teachers with essential\n\nvarious\n\ncurriculum\n\npedagogical skills. AASTU mandates terminal degrees for its\n\ndevelopment, accreditation, teacher capacity building,\n\nteaching staff, fostering a culture of continuous learning. The\n\nand student personality development.\n\ninstitution promotes field externships, allowing teachers to\n\nSince 2019, AASTU has been meticulously designing\n\ngain practical insights into industry challenges and\n\nquality curricula with active involvement from\n\ndevelopments.\n\nstakeholders. The university is committed to meeting\n\nStudent Personality Development: AASTU goes beyond class\n\ninternational standards in engineering programs\n\nroom education by nurturing students' overall development.\n\n(Washington Accord) and applied and natural science\n\nThe Student Career Development Unit and Student\n\nprograms\n\norganizations).",0.456330
2,"Delivering world-class education and training in strategically prioritized science and technology disciplines based on national economic demand.\n\nConducting problem-solving applied researches to support the productivity and competitiveness of industries.\n\nServing as a center for knowledge and technological adaptation, innovation and transfer.\n\nBuilding the technical and managerial capabilities of industries.\n\nBuilding a national hub of science and technology.\n\n1.4 AASTU Organization Governing and advisory bodies of the University consist of: a) Board; b) President; c) Senate; d) Managing Council; e) University Council; f) University Advisory Council; g) College Council; h) Academic Unit Managing Council; i) Department Assembly; j) Advisory, Standing or Ad-Hoc committee’s\n\n4\n\nAASTU Student Handbook 2023/2024",0.442357


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-neox-20b")

In [29]:
# # Get the eos_token_id
# eos_token_id = tokenizer.eos_token_id

# # Set pad_token_id to eos_token_id
# model.config.pad_token_id = eos_token_id


In [47]:
text_generator = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
    max_length=1000,
    do_sample=False,
    repetition_penalty=1.6,
    temperature=0.3,
    top_k=3
)

In [48]:
from langchain_huggingface import HuggingFacePipeline

# Create HuggingFacePipeline LLM
llm = HuggingFacePipeline(pipeline=text_generator)


In [49]:
# Supress warnings
warnings.filterwarnings("ignore")

prompt = """Use the given context to answer the question.
If you don't know the answer, just say that you don't know. Never Hallucinate or Repeat the answers.
Keep the answer as concise as possible.

Context: {context}

Question: {question}
Answer:
"""

qa_prompt_template = PromptTemplate.from_template(prompt)

In [50]:
# Create the LLM Chain
llm_chain = LLMChain(
    llm=llm,
    prompt=qa_prompt_template
)

In [51]:
# Create the StuffDocumentsChain
stuff_docs_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"  # Specify the name of the variable that will store the retrieved documents
)

In [52]:
# Combine into RetrievalQA Chain
qa = RetrievalQA(
    retriever=db.as_retriever(search_kwargs={'k': 1}),
    combine_documents_chain=stuff_docs_chain
)

In [ ]:
# Perform retrieval Q&A
qa_response = qa({"query": "Civil Engineering Courses in AASTU"})
Markdown(qa_response["result"])

In [ ]:
import time
# Perform retrieval Q&A
start_time = time.time()  # Start time measurement
qa_response = qa({"query": "Civil Engineering Courses in AASTU"})
search_time = time.time() - start_time  # End time measurement - measure search time

# Print the retrieved documents (optional)
print(f"Retrieved documents: {qa_response['result']}")

# Print the answer and search time
Markdown(qa_response["result"])
print(f"FAISS Search Time: {search_time:.2f} seconds")


In [37]:
# def query_chatbot(question: str) -> str:
#     try:
#         result = qa.run(question)
#         # Extract the answer from the result
#         answer = result.split("Helpful Answer:")[1].strip()
#         return answer
#     except Exception as e:
#         return f"An error occurred: {str(e)}"


KeyboardInterrupt



In [ ]:
# with gr.Blocks() as demo:
#     gr.Markdown("AASTUChat")
#     question_input = gr.Textbox(label="Ask a Question")
#     answer_output = gr.Textbox(label="Answer", interactive=False)
#     submit_button = gr.Button("Submit")
#     submit_button.click(fn=query_chatbot, inputs=question_input, outputs=answer_output)

# # Launch Gradio App
# demo.launch(share=True)

In [ ]:
# import requests
# from time import sleep

# # Replace 'YOUR_GRADIO_URL' with the actual URL of your Gradio interface
# keepalive_url = f'{127.0.0.1:7860}/keepalive'

# while True:
#   # Send a GET request to the keepalive endpoint every 5 minutes
#   requests.get(keepalive_url)
#   sleep(300)  # Wait 5 minutes between requests
